In [2]:
import pandas as pd
import re

#puxando dados do banco central
def consulta_bc(codigo_bcb):
  url3 = 'http://api.bcb.gov.br/dados/serie/bcdata.sgs.{}/dados?formato=json'.format(codigo_bcb)
  df_consulta = pd.read_json(url3)
  df_consulta.rename(columns={'data': 'Data'}, inplace=True)
  df_consulta['Data'] = pd.to_datetime(df_consulta['Data'], dayfirst=True)
  df_consulta.set_index('Data', inplace=True)
  return df_consulta

cdi = consulta_bc(12) #12 é o código para o CDI no site do BC
cdi.reset_index(inplace=True)
cdi['Data']= cdi['Data'].dt.strftime('%Y-%m-%d') #colocando data no padrao brasileiro

# Definindo a taxa anual adicional
taxa_anual_adicional = 0.03  # 3%

# Exemplo de um fator diário do CDI
fator_diario_cdi = cdi[cdi['Data'] == '2024-03-21']['valor'].iloc[0]
#fator_diario_cdi = 0.0265  # Este valor seria obtido do DataFrame cdi_df


# Convertendo a taxa do CDI para uma "cota"
cota_cdi_atual = 1 * (1 + (fator_diario_cdi / 100))

# A cota do dia anterior (considerada como 1.0 para simplificação)
cota_cdi_anterior = 1.0

# Convertendo a taxa anual adicional para uma base diária
taxa_diaria_adicional = (1 + taxa_anual_adicional) ** (1/252) - 1

# Ajustando a cota atual pelo CDI + 3% ao ano
cota_ajustada = cota_cdi_atual * (1 + taxa_diaria_adicional)

# Calculando a rentabilidade diária
rentabilidade_diaria = (cota_ajustada / cota_cdi_anterior - 1) * 100

print(f'Rentabilidade diária: {rentabilidade_diaria:.2f}%')


Rentabilidade diária: 0.05%
